In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from cv2 import imread, imwrite
import os
import glob
from tensorflow.image import psnr as psnr
from tensorflow.keras import datasets, layers, models
import matplotlib.image as mpimg
from matplotlib.pyplot import savefig
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets.mnist import load_data
from numpy import reshape
import matplotlib.pyplot as plt
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave, imread, imshow
from skimage.transform import resize
from skimage import color

import random

In [ ]:
path_airplane = 'natural_images\\airplane\\*.jpg'
path_car = 'natural_images\\car\\*.jpg'
path_cat = 'natural_images\\cat\\*.jpg'
path_dog = 'natural_images\\dog\\*.jpg'
path_flower = 'natural_images\\flower\\*.jpg'
path_fruit = 'natural_images\\fruit\\*.jpg'
path_motorbike = 'natural_images\\motorbike\\*.jpg'
path_person = 'natural_images\\person\\*.jpg'
paths_list = [path_airplane, path_car, path_cat, path_dog, path_flower, path_fruit,  path_motorbike, path_person]

In [ ]:
acum = 1
for path in paths_list:
    filenames = glob.glob(path)
    for filename in filenames:
        image = cv2.imread(filename)
        gray_img = cv2.imread(filename, 0)
        image = cv2.resize(image, (256, 256))
        gray_img = cv2.resize(gray_img, (256, 256))
        cv2.imwrite("gray_images/gray_" +str(acum) +".jpg", gray_img)
        cv2.imwrite("color_images/color_" +str(acum) +".jpg", image)
        acum += 1

In [8]:
path_quality = 'quality\\*.jpg'
paths_list = [path_quality]

In [10]:
acum = 1
for path in paths_list:
    filenames = glob.glob(path)
    for filename in filenames:
        #image = cv2.imread(filename)
        gray_img = cv2.imread(filename, 0)
        #image = cv2.resize(image, (1024, 1024))
        gray_img = cv2.resize(gray_img, (1024, 1024))
        cv2.imwrite("quality_gray/quality_gray_" +str(acum) +".jpg", gray_img)
        #cv2.imwrite("color_images/color_" +str(acum) +".jpg", image)
        acum += 1

In [3]:
# Get images
X = []
for filename in os.listdir('quality/'):
    X.append(img_to_array(load_img('quality/'+filename)))
X = np.array(X, dtype=float)

# Set up training and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

#Design the neural network
model = Sequential()
model.add(InputLayer(input_shape=(1024, 1024, 1)))
model.add(Dense(256, activation = 'relu')),
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

# Finish model
model.compile(optimizer='rmsprop', loss='mse')

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 2
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024, 1024, 256)   512       
_________________________________________________________________
conv2d (Conv2D)              (None, 1024, 1024, 64)    147520    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 64)      36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 512, 512, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 128)     147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 256, 256)     295168    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 256)     5

In [7]:
# Train model
TensorBoard(log_dir='/callback')
model.fit(image_a_b_gen(batch_size), steps_per_epoch=5, epochs=20)
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))


Epoch 1/10


In [7]:
# Load black and white images
color_me = []
for filename in os.listdir('test256/'):
        color_me.append(img_to_array(load_img('test256/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [9]:
# Test model
output = model.predict(color_me)
output = output * 128


# Output colorizations
for i in range(len(output)):
        cur = np.zeros((256, 256, 3))
        cur[:,:,0] = color_me[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave("result4/img_"+str(i)+".png", lab2rgb(cur))

arning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. 

in_hsv_h = color.convert_colorspace(output, 'RGB', 'HSV')
in_hsv_s = in_hsv_h.copy()
in_hsv_v = in_hsv_h.copy()

for i in range(newImage.shape[0]):
    in_hsv_h[i,:,0] = np.sort(in_hsv_h[i,:,0])
    in_hsv_s[i,:,1] = np.sort(in_hsv_s[i,:,1])
    in_hsv_v[i,:,2] = np.sort(in_hsv_v[i,:,2])

imsave('testing-sorted-hue.png', (color.convert_colorspace(in_hsv_h, 'HSV', 'RGB')*255).astype(np.uint8))
imsave('testing-sorted-saturation.png', (color.convert_colorspace(in_hsv_s, 'HSV', 'RGB')*255).astype(np.uint8))

In [1]:
model_json = model.to_json()
with open('model_browns.json', 'w') as json_file:
    json_file.write(model_json)

model.save('model_browns.h5')

NameError: name 'model' is not defined